# Prédire le risque de surentraînement
## Objectif : Utiliser les caractéristiques instantanées pour estimer si cette personne est à risque de surentraînement ou non, en se basant sur des corrélations entre les variables.
### Variables clés à analyser :
- **Resting_BPM** : Un Resting_BPM élevé peut indiquer un stress physiologique.
- **Fat_Percentage** et BMI : Un déséquilibre peut suggérer un métabolisme perturbé.
- **Workout_Frequency et Session_Duration** : Une fréquence ou durée excessive peut être un facteur de risque.
- **Calories_Burned** : Un nombre anormalement bas ou élevé par rapport à la moyenne du groupe.
- **Experience_Level** : Les débutants et les athlètes expérimentés n’ont pas les mêmes risques.

### Import

In [128]:
import pandas as pd
import numpy as np

### Consolidation des datas
Nous avons 2 datasets qui possèdent les mêmes colonnes, nous allons les unir pour en former qu'un et en faire un csv.

In [129]:
df1 = pd.read_csv('exercise_tracking.csv')
df1.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39


In [130]:
df2 = pd.read_csv('exercise_tracking_synthetic_data.csv')
df2.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,34.0,Female,86.7,1.86,174,152.0,74.0,1.12,712.0,Strength,12.8,2.4,5.0,2.0,14.31
1,26.0,Female,84.7,1.83,166,156.0,73.0,1.00,833.0,Strength,27.9,2.8,5.0,2.0,33.49
2,22.0,Male,64.8,1.85,187,166.0,64.0,1.24,1678.0,Cardio,28.7,1.9,3.0,2.0,12.73
3,54.0,Female,75.3,1.82,187,169.0,58.0,1.45,628.0,Cardio,31.8,2.4,4.0,1.0,20.37
4,34.0,Female,52.8,1.74,177,169.0,66.0,1.60,1286.0,Strength,26.4,3.2,4.0,2.0,20.83


In [131]:
df1.shape, df2.shape

((973, 15), (1800, 15))

In [132]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            973 non-null    int64  
 1   Gender                         973 non-null    object 
 2   Weight (kg)                    973 non-null    float64
 3   Height (m)                     973 non-null    float64
 4   Max_BPM                        973 non-null    int64  
 5   Avg_BPM                        973 non-null    int64  
 6   Resting_BPM                    973 non-null    int64  
 7   Session_Duration (hours)       973 non-null    float64
 8   Calories_Burned                973 non-null    float64
 9   Workout_Type                   973 non-null    object 
 10  Fat_Percentage                 973 non-null    float64
 11  Water_Intake (liters)          973 non-null    float64
 12  Workout_Frequency (days/week)  973 non-null    int

In [133]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            1790 non-null   float64
 1   Gender                         1729 non-null   object 
 2   Weight (kg)                    1778 non-null   float64
 3   Height (m)                     1774 non-null   float64
 4   Max_BPM                        1779 non-null   object 
 5   Avg_BPM                        1770 non-null   float64
 6   Resting_BPM                    1781 non-null   float64
 7   Session_Duration (hours)       1777 non-null   float64
 8   Calories_Burned                1777 non-null   float64
 9   Workout_Type                   1739 non-null   object 
 10  Fat_Percentage                 1784 non-null   float64
 11  Water_Intake (liters)          1776 non-null   float64
 12  Workout_Frequency (days/week)  1742 non-null   f

In [134]:
# Convertir les types de df2 en type de df1 car c'est les plus cohérentes
for col in df1.columns:
    dtype = df1[col].dtype
    if dtype == 'int64':
        df2[col] = pd.to_numeric(df2[col], errors='coerce').fillna(0).astype('int64') # on forme la conversion en cas d'erreur
    elif dtype == 'float64':
        df2[col] = pd.to_numeric(df2[col], errors='coerce').fillna(0.0)
    else:
        df2[col] = df2[col].astype(dtype)

# Vérifier les types
print(df2.dtypes)

Age                                int64
Gender                            object
Weight (kg)                      float64
Height (m)                       float64
Max_BPM                            int64
Avg_BPM                            int64
Resting_BPM                        int64
Session_Duration (hours)         float64
Calories_Burned                  float64
Workout_Type                      object
Fat_Percentage                   float64
Water_Intake (liters)            float64
Workout_Frequency (days/week)      int64
Experience_Level                   int64
BMI                              float64
dtype: object


In [135]:
df = pd.concat([df1, df2], ignore_index=True)
df.shape

(2773, 15)

In [136]:
df.to_csv('consolidated_exercise_tracking.csv', index=False)

## Création de la variable cible : Risque de surentraînement

### Objectif
Créer une colonne binaire `Overtraining_Risk` (0 = faible risque, 1 = risque élevé) en se basant sur des valeurs de référence scientifiques pour évaluer si un individu présente des signes de surentraînement.

### Méthodologie

#### Étape 1 : Construction des données de référence
Nous avons créé des tableaux de valeurs normales ajustées selon l'âge et le sexe, basés sur des sources médicales reconnues :

**Sources utilisées :**
- **Fréquence cardiaque au repos** : American Heart Association (AHA) 2024, National Institutes of Health (NIH) 2023
- **Pourcentage de masse grasse** : American Council on Exercise (ACE) 2024
- **Fréquence cardiaque maximale** : Formule de Tanaka et al., Journal of the American College of Cardiology (JACC) 2001
- **Indice de Masse Corporelle (IMC)** : Organisation Mondiale de la Santé (OMS) 2024

#### Étape 2 : Calcul du score de risque
Pour chaque individu, nous calculons un score de risque basé sur plusieurs indicateurs physiologiques :

1. **Fréquence cardiaque au repos élevée** : Un resting BPM supérieur aux valeurs normales pour l'âge indique un stress physiologique (principal indicateur de surentraînement)

2. **Réserve cardiaque faible** : Différence entre FC max et FC repos < seuils attendus (signe de fatigue cardiovasculaire)

3. **Composition corporelle déséquilibrée** : 
   - Masse grasse trop basse (< seuil essentiel) -> récupération compromise
   - Masse grasse trop élevée (> seuil obésité) -> stress métabolique

4. **Charge d'entraînement excessive** : Fréquence * durée des séances trop élevée

5. **Durée de séance prolongée** : Sessions > 2h augmentent le risque de fatigue

6. **Hydratation insuffisante** : Apport en eau < 2L compromet la récupération

7. **IMC anormal** : Maigreur ou obésité amplifient le stress physiologique

8. **Efficacité calorique faible** : Calories brûlées/heure < seuils -> signe de fatigue

9. **Niveau d'expérience** : Les débutants sont plus vulnérables

#### Étape 3 : Classification finale
- **Score de risque ≥ 7** -> `Overtraining_Risk = 1` (Risque ÉLEVÉ)
- **Score de risque 4-6** -> `Overtraining_Risk = 1` (Risque MODÉRÉ)
- **Score de risque < 4** -> `Overtraining_Risk = 0` (Risque FAIBLE)

---

### Références complètes

1. **American Heart Association (AHA).** "Target Heart Rates Chart." 2024. 
   https://www.heart.org/en/healthy-living/fitness/fitness-basics/target-heart-rates

2. **National Institutes of Health (NIH).** "Resting Heart Rate Reference Data." National Health Statistics Reports. 2023.

3. **American Council on Exercise (ACE).** "Body Fat Percentage Norms." 2024.

4. **Tanaka H, Monahan KD, Seals DR.** "Age-predicted maximal heart rate revisited." 
   Journal of the American College of Cardiology. 2001;37(1):153-156.

5. **Organisation Mondiale de la Santé (OMS).** "Classification de l'IMC." 2024.

In [137]:
df.Age.min(), df.Age.max()

(0, 59)

In [138]:
ages = df.Age.unique()
ages.sort()
ages

array([ 0, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59])

In [139]:
# ============================================================================
# RÉFÉRENCES : Valeurs normales selon âge et sexe
# Sources : AHA 2024, NIH 2023, ACE 2024
# ============================================================================

# 1. Fréquence cardiaque au repos (BPM) selon l'âge
resting_hr_reference = pd.DataFrame({
    'Age_Min': [18, 26, 36, 46, 56, 66],
    'Age_Max': [25, 35, 45, 55, 65, 100],
    'Normal_Min': [60, 61, 62, 63, 64, 65],
    'Normal_Max': [80, 80, 81, 82, 83, 84],
    'Athlete_Min': [40, 41, 42, 43, 44, 45],
    'Athlete_Max': [54, 55, 56, 57, 58, 60]
})

# 2. Pourcentage de masse grasse (Body Fat %) selon le sexe et l'âge
body_fat_reference = pd.DataFrame({
    'Sexe': ['Male', 'Male', 'Male', 'Male', 'Female', 'Female', 'Female', 'Female'],
    'Age_Min': [18, 30, 40, 50, 18, 30, 40, 50],
    'Age_Max': [29, 39, 49, 100, 29, 39, 49, 100],
    'Essentiel_Max': [5, 5, 5, 5, 13, 13, 13, 13],
    'Athlète_Min': [6, 6, 7, 8, 14, 15, 16, 17],
    'Athlète_Max': [13, 14, 15, 16, 21, 22, 23, 24],
    'Fitness_Max': [17, 18, 19, 20, 25, 26, 27, 28],
    'Acceptable_Max': [24, 25, 26, 27, 31, 32, 33, 34],
    'Obésité_Seuil': [25, 26, 27, 28, 32, 33, 34, 35]
})

# 3. Fréquence cardiaque maximale théorique (formule Tanaka et al., JACC 2001)
def get_max_hr_expected(age):
    return 208 - (0.7 * age)

# 4. IMC de référence selon OMS 2024
bmi_reference = {
    'Maigreur': 18.5,
    'Normal': 24.9,
    'Surpoids': 29.9,
    'Obésité classe I': 34.9,
    'Obésité classe II': 39.9,
    'Obésité classe III': 40.0
}

In [140]:
def get_resting_hr_range(age):
    """Obtenir la plage normale de fréquence cardiaque au repos pour l'age en paramètre"""
    ref = resting_hr_reference[
        (resting_hr_reference['Age_Min'] <= age) & 
        (resting_hr_reference['Age_Max'] >= age)
    ]
    if len(ref) > 0:
        return ref.iloc[0]['Normal_Min'], ref.iloc[0]['Normal_Max']
    
    # Si on n'a pas trouvé l'age dans le tableau de référence
    if age < 18:
        return 70, 100
    elif age > 100:
        return 65, 100

def get_body_fat_range(age, gender):
    """Get normal body fat range for age and gender"""
    ref = body_fat_reference[
        (body_fat_reference['Sexe'] == gender) &
        (body_fat_reference['Age_Min'] <= age) & 
        (body_fat_reference['Age_Max'] >= age)
    ]
    if len(ref) > 0:
        return (ref.iloc[0]['Essentiel_Max'], ref.iloc[0]['Acceptable_Max'], ref.iloc[0]['Obésité_Seuil'])
    
    if gender == 'Male':
        return 5, 21, 28
    else :
        return 13, 20, 32

In [141]:
def calculate_overtraining_risk(row):
    """
    Calcule le risque de surentraînement en fonction des valeurs de référence ajustées en fonction de l'âge et du sexe
    Returns: risk_score (int), risk_level (str), details (dict)
    """
    risk_score = 0
    details = {}
    
    age = row['Age']
    gender = row['Gender']
    
    # -------------------------------------------------------------------------
    # 1. ANALYSE DU BPM AU REPOS 
    # -------------------------------------------------------------------------
    hr_min, hr_max = get_resting_hr_range(age)
    resting_bpm = row['Resting_BPM']
    
    if resting_bpm > hr_max + 10:
        risk_score += 3  # Sévèrement élevé
        details['Resting_HR'] = f"Severely elevated ({resting_bpm} vs normal {hr_min}-{hr_max})"
    elif resting_bpm > hr_max + 5:
        risk_score += 2  # Modérément élevé
        details['Resting_HR'] = f"Elevated ({resting_bpm} vs normal {hr_min}-{hr_max})"
    elif resting_bpm > hr_max:
        risk_score += 1  # Légèrement élevé
        details['Resting_HR'] = f"Slightly elevated ({resting_bpm} vs normal {hr_min}-{hr_max})"
    else:
        details['Resting_HR'] = "Normal"
    
    # -------------------------------------------------------------------------
    # 2. BPM RESERVE (Max BPM - BPM au repos)
    # -------------------------------------------------------------------------
    expected_max_hr = get_max_hr_expected(age)
    hr_reserve = row['Max_BPM'] - row['Resting_BPM']
    expected_reserve = expected_max_hr - hr_max
    
    # Une faible réserve cardiaque indique une mauvaise condition physique cardiovasculaire ou un surentraînement
    if hr_reserve < expected_reserve * 0.7:  # Moins que 70% attendu
        risk_score += 2
    elif hr_reserve < expected_reserve * 0.85:
        risk_score += 1
    
    # -------------------------------------------------------------------------
    # 3. POURCENTAGE DU BODY FAT 
    # -------------------------------------------------------------------------
    essential, normal, high = get_body_fat_range(age, gender)
    fat_pct = row['Fat_Percentage']
    
    if fat_pct < essential:
        risk_score += 2  # Too low - hormonal issues, poor recovery
    elif fat_pct > high:
        risk_score += 1  # Too high - metabolic stress
    elif fat_pct > normal:
        risk_score += 0.5
    
    # -------------------------------------------------------------------------
    # 4. VOLUME D'ENTRAÎNEMENT (Fréquence * Durée)
    # -------------------------------------------------------------------------
    workout_freq = row['Workout_Frequency (days/week)']
    session_duration = row['Session_Duration (hours)']
    training_load = workout_freq * session_duration
    
    if training_load > 12:  # ex: 6j * 2h
        risk_score += 3
    elif training_load > 10:
        risk_score += 2
    elif training_load > 8:
        risk_score += 1
    # Une seule séance très longue
    if session_duration > 2.5:
        risk_score += 1
    elif session_duration > 2.0:
        risk_score += 0.5
    
    # -------------------------------------------------------------------------
    # 5. INDICATEURS DE RÉCUPÉRATION
    # -------------------------------------------------------------------------
    # Eau
    water_intake = row['Water_Intake (liters)']
    if water_intake < 2.0:
        risk_score += 1
    elif water_intake < 2.5:
        risk_score += 0.5
    
    # -------------------------------------------------------------------------
    # 6. NIVEAU D'EXPÉRIENCE
    # -------------------------------------------------------------------------
    experience = row['Experience_Level']
    
    # Les débutants sont plus susceptibles de souffrir de surentraînement
    if experience == 1:  # Débutant
        risk_score *= 1.3  # augmentation du risque de 30%
    elif experience == 2:  # Intermédiaire
        risk_score *= 1.1  # augmentation de 10%
    
    # -------------------------------------------------------------------------
    # 7. CONSIDÉRATION DE L'IMC (selon OMS 2024)
    # -------------------------------------------------------------------------
    bmi = row['BMI']

    # Maigreur
    if bmi < bmi_reference['Maigreur']:
        risk_score += 2  # Risque élevé : récupération compromise, déficit énergétique
        
    # Normal (pas de pénalité)

    # Surpoids (léger risque)
    elif bmi <= bmi_reference['Surpoids']:
        risk_score += 1  # Charge métabolique et articulaire accrue

    # Obésité classe I (risque modéré)
    elif bmi <= bmi_reference['Obésité classe I']:
        risk_score += 2

    # Obésité classe II (risque élevé)
    elif bmi <= bmi_reference['Obésité classe II']:
        risk_score += 3

    # Obésité classe III (risque très élevé)
    else:
        risk_score += 4
    
    # -------------------------------------------------------------------------
    # 8. EFFICACITÉ CALORIQUE
    # -------------------------------------------------------------------------
    calories_burned = row['Calories_Burned']
    calories_per_hour = calories_burned / session_duration if session_duration > 0 else 0
    
    # Une très faible dépense calorique peut indiquer de la fatigue ou une inefficacité
    if calories_per_hour < 250:
        risk_score += 1
    
    # -------------------------------------------------------------------------
    # CLASSIFICATION FINALE
    # -------------------------------------------------------------------------
    if risk_score >= 7:
        risk_level = "HIGH"
        risk_binary = 1
    elif risk_score >= 4:
        risk_level = "MODERATE"
        risk_binary = 1
    else:
        risk_level = "LOW"
        risk_binary = 0
    
    return risk_binary, risk_level, risk_score

In [142]:
# ============================================================================
# APPLY TO DATAFRAME
# ============================================================================

def add_overtraining_risk_column(df):
    """
    Ajouter des colonnes de risque de surentraînement au dataframe.
    
    Paramètres :
    df : DataFrame avec les colonnes requises.
    
    Résultats :
    DataFrame avec les colonnes ajoutées :
        - Overtraining_Risk (binaire : 0 ou 1)
        - Risk_Level (LOW, MODERATE, HIGH)
        - Risk_Score (score continu)
        - Risk_Details (dictionnaire avec explication)
    """
    results = df.apply(calculate_overtraining_risk, axis=1, result_type='expand')
    results.columns = ['Overtraining_Risk', 'Risk_Level', 'Risk_Score']
    
    df_with_risk = pd.concat([df, results], axis=1)
    
    return df_with_risk

In [143]:
new_df = add_overtraining_risk_column(df)
new_df

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI,Overtraining_Risk,Risk_Level,Risk_Score
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20,0,LOW,2.00
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00,0,LOW,3.10
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71,0,LOW,2.65
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41,0,LOW,3.95
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39,0,LOW,3.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2768,54,Male,88.5,2.00,173,134,58,1.11,1388.0,HIIT,27.7,3.7,3,2,36.73,0,LOW,3.55
2769,52,Male,84.3,1.69,164,169,54,0.77,1367.0,HIIT,32.6,2.9,3,2,15.11,0,LOW,3.10
2770,47,Male,70.1,1.84,188,129,67,1.20,1261.0,Strength,28.4,2.5,3,2,17.99,0,LOW,3.10
2771,35,Male,49.3,1.71,180,152,73,1.04,956.0,Cardio,32.9,1.7,4,3,12.65,1,MODERATE,4.00


In [144]:
new_df[new_df['Risk_Level'] == 'LOW'].shape, new_df[new_df['Risk_Level'] == 'MODERATE'].shape, new_df[new_df['Risk_Level'] == 'HIGH'].shape

((2542, 18), (228, 18), (3, 18))

**Remarque:** Il faudra utiliser un moyen d'augmenter le nombre de MODERATE et HIGH pour éviter :
- Overfitting sur la classe LOW : le modèle va apprendre à prédire "LOW" tout le temps
- Underfitting sur HIGH : avec 3 exemples, le modèle ne peut pas apprendre à prédire cette classe


Pour y remédier nous allons dans un premier temps, simplement rendre la classe binaire : 0 = LOW & 1 = MODERATE+HIGH 

In [145]:
# Regrouper MODERATE et HIGH ensemble
new_df['Overtraining_Risk_Binary'] = new_df['Risk_Level'].apply(
    lambda x: 0 if x == 'LOW' else 1
)

print(new_df['Overtraining_Risk_Binary'].value_counts())

Overtraining_Risk_Binary
0    2542
1     231
Name: count, dtype: int64


In [147]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

X = new_df.drop(['Overtraining_Risk_Binary', 'Risk_Level', 'Risk_Score', 'Overtraining_Risk'], axis=1)
y = new_df['Overtraining_Risk_Binary']

X_encoded = pd.get_dummies(X, columns=['Gender', 'Workout_Type'])

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

# Modèle SANS correction
model_baseline = LogisticRegression(max_iter=1000, random_state=42)
model_baseline.fit(X_train, y_train)
y_pred_baseline = model_baseline.predict(X_test)

print("=== MODÈLE SANS CORRECTION ===")
print(classification_report(y_test, y_pred_baseline, target_names=['Low Risk', 'At Risk']))
print("\nMatrice de confusion :")
print(confusion_matrix(y_test, y_pred_baseline))

=== MODÈLE SANS CORRECTION ===
              precision    recall  f1-score   support

    Low Risk       0.93      1.00      0.96       509
     At Risk       0.78      0.15      0.25        46

    accuracy                           0.93       555
   macro avg       0.85      0.57      0.61       555
weighted avg       0.92      0.93      0.90       555


Matrice de confusion :
[[507   2]
 [ 39   7]]


/opt/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
